In [1]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from glob import glob
import multiprocessing
import pandas as pd
from tqdm import tqdm
import time
from copy import deepcopy
from multiprocessing import Pool
from hydra import initialize, compose
import pickle

from package.helpers.test_physical_lens import TestPhysicalLens
from package.helpers.lens import Lens
from package.plots import diagnostic_plot, plot
from package.utils import util
from package.scripts import generate
from package.helpers import pyhalo


In [3]:
def generate_lens(row):
    return Lens(z_lens=row['redslens'], 
                z_source=row['redssour'], 
                theta_e=row['angleins'], 
                lens_x=row['xposlens'], 
                lens_y=row['yposlens'], 
                source_x=row['xpossour'], 
                source_y=row['ypossour'], 
                mag_lens=row['magtlensF106'], 
                mag_source=row['magtsourF106'])

In [4]:
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, repo_dir, pickle_dir = config.machine.array_dir, config.machine.data_dir, config.machine.repo_dir, config.machine.pickle_dir
util.create_directory_if_not_exists(pickle_dir)

In [5]:
# get output of SkyPy pipeline
df = pd.read_csv(os.path.join('/data','bwedig', 'roman-population', 'data', 'dictparaggln_Area00000010.csv'))

lens_list = []

In [6]:
# split the rows into batches based on core count
cpu_count = multiprocessing.cpu_count()
process_count = int(cpu_count / 2)  # TODO consider making this larger, but using all CPUs has crashed
generator = util.batch_list(list(df.iterrows()), process_count)
batches = list(generator)

In [8]:
# generate the lens objects
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    lens = Lens(z_lens = row['redslens'], 
                z_source=row['redssour'], 
                sigma_v=row['velodisp'], 
                lens_x=row['xposlens'], 
                lens_y=row['yposlens'], 
                source_x=row['xpossour'], 
                source_y=row['ypossour'], 
                mag_lens=row['magtlensF106'], 
                mag_source=row['magtsourF106'])
    
    lens_list.append(lens)

TypeError: object of type 'generator' has no len()

In [ ]:
# pickle lens list
with open(os.path.join(pickle_dir, 'skypy_output_lens_list'), 'ab') as results_file:
    pickle.dump(lens_list, results_file)